<a href="https://colab.research.google.com/github/AdhamAymanAlrfae/malaria-parasitized-and-uninfected-cells-/blob/main/Malaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D,MaxPool2D,BatchNormalization,Flatten,Dense,Resizing,Dropout,Normalization

In [ ]:
data,data_info = tfds.load(
    'malaria',
    split='train',
    as_supervised = True,
    shuffle_files=True,
    batch_size = 32,
    with_info=True
    )

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/27558 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/malaria/1.0.0.incompleteUYO56W/malaria-train.tfrecord*...:   0%|          …

Dataset malaria downloaded and prepared to /root/tensorflow_datasets/malaria/1.0.0. Subsequent calls will reuse this data.


In [ ]:
def splet(DATA,TRAIN_SIZE,TEST_SIZE,VAL_SIZE=0):
  DATA_SIZE = len(DATA)
  train = DATA.take(int(DATA_SIZE*TRAIN_SIZE))
  test_val = DATA.skip(int(DATA_SIZE*TRAIN_SIZE))
  val = test_val.take(int(DATA_SIZE*VAL_SIZE))
  test = test_val.skip(int(DATA_SIZE*VAL_SIZE))
  return train,test,val

In [ ]:
train,test,val = splet(data,0.8,0.1,0.1)

In [ ]:
print(len(data))
print(len(train))
print(len(test))
print(len(val))

862
689
87
86


In [ ]:
class MnistModel(Model):
  def __init__(self) -> None:
    super().__init__()
    self.resize = Resizing(240,240)
    self.conv_1 = Conv2D(32, (3, 3), activation='relu', input_shape=(240, 240, 3))
    self.conv_2 = Conv2D(16, (3, 3), activation='relu', input_shape=(240, 240, 3))
    self.max_1 = MaxPool2D(pool_size=(2, 2))
    self.max_2= MaxPool2D(pool_size=(2, 2))
    self.flatten = Flatten()
    self.dense_1 = Dense(128,activation="relu",kernel_regularizer='l2')
    self.dense_2 = Dense(64,activation="relu",kernel_regularizer='l2')
    self.dense_3 = Dense(32,activation="relu",kernel_regularizer='l2')
    self.dense_4 = Dense(16,activation="relu",kernel_regularizer='l2')
    self.dense_out = Dense(1,activation="sigmoid")
    self.normalization_1 = BatchNormalization()
    self.normalization_2 = BatchNormalization()
    self.normalization_3 = BatchNormalization()
    self.normalization_4 = BatchNormalization()
    self.normalization_5 = BatchNormalization()
    self.normalization_6 = BatchNormalization()
    self.dropout_1 = Dropout(0.3)
    self.dropout_2 = Dropout(0.4)


  def call(self, inputs):
        x = self.resize(inputs)
        x = self.conv_1(x)
        x = self.normalization_1(x)
        x = self.max_1(x)
        x = self.conv_2(x)
        x = self.normalization_2(x)
        x = self.max_2(x)
        x = self.flatten(x)
        x = self.dense_1(x)
        x = self.dropout_1(x)
        x = self.normalization_3(x)
        x = self.dense_2(x)
        x = self.dropout_2(x)
        x = self.normalization_4(x)
        x = self.dense_3(x)
        x = self.normalization_5(x)
        x = self.dense_4(x)
        x = self.normalization_6(x)
        return self.dense_out(x)


In [ ]:
#Build the model
model = MnistModel()

model.build((None, 240, 240, 3))
model.summary()

Model: "mnist_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         multiple                  0         
                                                                 
 conv2d (Conv2D)             multiple                  896       
                                                                 
 conv2d_1 (Conv2D)           multiple                  4624      
                                                                 
 max_pooling2d (MaxPooling2  multiple                  0         
 D)                                                              
                                                                 
 max_pooling2d_1 (MaxPoolin  multiple                  0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           multiple                  

In [ ]:
#after 10 epoch the learn rate will be (e**-1)*LR after evry epoch
def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

In [ ]:
learn_rate_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)


In [ ]:
stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=4,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=10
)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history_model_3 = model.fit(train, epochs=40,validation_data=val,callbacks=[learn_rate_callback,stopping_callback])

Epoch 1/40
689/689 [==============================] - 62s 66ms/step - loss: 2.6650 - accuracy: 0.5975 - val_loss: 1.9606 - val_accuracy: 0.5073 - lr: 0.0010
Epoch 2/40
689/689 [==============================] - 55s 80ms/step - loss: 1.3639 - accuracy: 0.8289 - val_loss: 1.0652 - val_accuracy: 0.9157 - lr: 0.0010
Epoch 3/40
689/689 [==============================] - 45s 65ms/step - loss: 1.2365 - accuracy: 0.8849 - val_loss: 1.6459 - val_accuracy: 0.8761 - lr: 0.0010
Epoch 4/40
689/689 [==============================] - 46s 67ms/step - loss: 1.3392 - accuracy: 0.9045 - val_loss: 1.3119 - val_accuracy: 0.8953 - lr: 0.0010
Epoch 5/40
689/689 [==============================] - 46s 67ms/step - loss: 1.3748 - accuracy: 0.9143 - val_loss: 1.4606 - val_accuracy: 0.9273 - lr: 0.0010
Epoch 6/40
689/689 [==============================] - 46s 66ms/step - loss: 1.2459 - accuracy: 0.9212 - val_loss: 1.4124 - val_accuracy: 0.9462 - lr: 0.0010
Epoch 7/40
689/689 [==============================] - 46s 

In [ ]:
model.evaluate(test)

87/87 [==============================] - 13s 18ms/step - loss: 0.2834 - accuracy: 0.9503


[0.2833837866783142, 0.9503263235092163]